In [27]:
import glob
from pathlib import Path
from natsort import natsorted
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.layout import LAParams
from io import StringIO
import img2pdf
import os
from tqdm.notebook import tqdm
import numpy as np

In [9]:
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import (
    LAParams,
    LTContainer,
    LTTextLine,
)

def get_objs(layout, results):
    if not isinstance(layout, LTContainer):
        return
    for obj in layout:
        if isinstance(obj, LTTextLine):
            results.append({'bbox': obj.bbox, 'text' : obj.get_text(), 'type' : type(obj)})
        get_objs(obj, results)

def main(path):
    with open(path, "rb") as f:
        parser = PDFParser(f)
        document = PDFDocument(parser)
        if not document.is_extractable:
            raise PDFTextExtractionNotAllowed
        # https://pdfminersix.readthedocs.io/en/latest/api/composable.html#
        laparams = LAParams(
            all_texts=True,
        )
        rsrcmgr = PDFResourceManager()
        device = PDFPageAggregator(rsrcmgr, laparams=laparams)
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        for page in PDFPage.create_pages(document):
            interpreter.process_page(page)
            layout = device.get_result()
            results = []
            print('objs-------------------------')
            get_objs(layout, results)
            for r in results:
                print(r)


main('hello2.pdf')

objs-------------------------
{'bbox': (90.744, 728.1928, 142.2056, 738.7528), 'text': 'Hello 世界 \n', 'type': <class 'pdfminer.layout.LTTextLineHorizontal'>}
{'bbox': (168.5, 728.1928, 223.8356, 738.7528), 'text': '猫が鳴いた \n', 'type': <class 'pdfminer.layout.LTTextLineHorizontal'>}
{'bbox': (168.5, 709.7128, 202.8356, 720.2728), 'text': '私は神 \n', 'type': <class 'pdfminer.layout.LTTextLineHorizontal'>}
{'bbox': (90.744, 691.1128, 146.0456, 701.6728), 'text': '神は死んだ \n', 'type': <class 'pdfminer.layout.LTTextLineHorizontal'>}
{'bbox': (168.5, 691.1128, 171.2456, 701.6728), 'text': ' \n', 'type': <class 'pdfminer.layout.LTTextLineHorizontal'>}
{'bbox': (168.5, 672.6328, 255.2756, 683.1928), 'text': 'あｗねおｌんｓｆ \n', 'type': <class 'pdfminer.layout.LTTextLineHorizontal'>}
{'bbox': (90.744, 709.7128, 93.4896, 720.2728), 'text': ' \n', 'type': <class 'pdfminer.layout.LTTextLineHorizontal'>}
{'bbox': (90.744, 672.6328, 93.4896, 683.1928), 'text': ' \n', 'type': <class 'pdfminer.layout.LTTextLineH

# ocr 見本

In [4]:
from PIL import Image
import sys
import pyocr
import pyocr.builders
import pdf2image

tools = pyocr.get_available_tools()
if len(tools) == 0:
    print("No OCR tool found")
    sys.exit(1)

# The tools are returned in the recommended order of usage
tool = tools[0]
print("Will use tool '%s'" % (tool.get_name()))
# Ex: Will use tool 'libtesseract'


# pdfから画像オブジェクトに
images = pdf2image.convert_from_path("output5.pdf",fmt='jpg')
# lang = 'eng'
lang = 'jpn'
# 画像オブジェクトからテキストに
for i,image in enumerate(images):
    txt = tool.image_to_string(
        image,
        lang=lang,
        builder=pyocr.builders.TextBuilder()
    )
#     if '地震'in txt:
#         print(i+1)
    print(txt)

Will use tool 'Tesseract (sh)'
ー 次の各問に答えなさい。 (24点)

問 ー 図ーは' ある地震のゅれを観測地点Aの地震計で記録したもので, 図ーの①と②は, それぞれ
P波の到着による小さなゆれの始まりとS波の到着による大きなゅれの始まりを示してぃます。 図2

は, この地震にっぃて, P波 ・ S波が到着するまでの時問と震源からの距離との関係を表したもの
です。 図ー と図2から' この地震の震源から観測地点Aまでの距離は何 km か求めなさぃ。 (3点)
① ②

土 *

 

S波

 
   

 

 

 

 

。 2 4 6 8 ]0 ー2 ]4 ー6 ー8 …言巨言巨巨 言 巨 菱 言 賣 言 言 言 言 言 茎 言 萱 妄 言 巨
観測地点Aで最初にゆれが記録 。 2 4 6 8 ー。 ー2 ー4 ー6 ー8 20 22

されたときからの時間〔秒〕 P波' S波が到着するまでの時間〔秒〕
図ー 図2

問 2 太陽の表面に見られる黒点の温度と して最も適切なものを, 次のァ~ェの中から一っ選ぴ,
その記号を書きなさぃ。 (3点〉
ア 約 50。 ”C イ 約 4。。。 竈 ウ 約 6。。。 む エ 約 ー600 万。C

問 3 図ー は, コリウスのふ入りの葉をスヶッチしたものです。 この葉の一部を' 図2のように
アルミニウムはくでおおって暗いところに一晩置き, 翌日, ひなたに置いて日光を十分に当て
ました。 アルミニウムはく をはずしてこの葉を熱湯にひたしだあと' 温めたェタノール中で
脱色し, ヨウ素液にひたしました。 ヨウ素液に反応し色が変化した部分を' 図3のア~エの中
から一っ選ぴ, その記号を書きなさぃ。 (3点)
せんたん けん ぴ きょう

問 4 次のA~Fは, タマネギの根の先端に近い部分の細胞を顕微鏡 で観察し, 細胞分裂の
いろいろな段階をスケッチしたものです。 Aを細胞分裂の最初, Fを細胞分裂の最後と し,

B~Eを細胞分裂の正しい順に並ぺかぇなさい。 〈3点)

 

問 5 次の表は, 6種類の物質のー気圧におけ る沸点と融点を調べてまとめたものです 。 ー気圧で 25。C
のときに液体である物質を, 表中のA~Fの中からすべて選ぴ, その記号を書きなさい。 (3

# ocr→txt 見本

In [122]:
images = pdf2image.convert_from_path(test,fmt='jpg')
for i,image in enumerate(images):
    txt = tool.image_to_string(
        image,
        lang=lang,
        builder=pyocr.builders.TextBuilder()
    )
    f = open(f'./intan/hokkaido/2021/test{i+1}.txt', 'w') 
    # 文字列を記載する
    f.write(txt) 
    # ファイルを閉じる
    f.close() 
#     if '地震'in txt:
#         print(i+1)

# 関数box

In [162]:
biology=['裸子植物と被子植物','植物の分類','動物の分類','細胞','光合成・蒸散・呼吸','根・茎','消化','呼吸','血液・排出','感覚と運動','細胞分裂','生殖','遺伝']

iden={'遺伝':['遺伝','細胞'],'細胞':['細','地球'],'消化':['消化酵素','消化']}

In [8]:
def search_(txtfile,bunya):
    f = open(txtfile, 'r')  
    if any(x in f.read() for x in iden[bunya]):
        a.append(txtfile)

def only_dir(path):
    return [i for i in os.listdir(path)if os.path.isdir(os.path.join(path,i))if not i.startswith('.')]
def only_file(path):
    return [i for i in os.listdir(path)if os.path.isfile(os.path.join(path,i))if not i.startswith('.')]
def specific_get_ext(_list,extension):
        return [i for i in _list if i.endswith(f'.{extension}') ]

def search_(bunya):
    if any(x in txt for x in iden[bunya]):
        print(bunya)

In [337]:
path='./intan'
a=[]
for pref in only_dir(path):
    c1=os.path.join(path,pref)
    for year in only_dir(c1):
        c2=os.path.join(c1,year)
        if sum(os.path.isfile(os.path.join(c2,name))for name in os.listdir(c2) if not name.startswith('.')if name.endswith('.txt'))>=1:
            for tfile in natsorted(specific_get_ext(only_file(c2),'txt')):
                search_(os.path.join(c2,tfile),'消化')

# pdfへ

In [ ]:
pdf_FileName = "{pref}_{year}.pdf" # 出力するPDFの名前
png_Folder = "./" # 画像フォルダ
extension  = ".jpg" # 拡張子がPNGのものを対象
with open(pdf_FileName,"wb") as f:
    # 画像フォルダの中にあるPNGファイルを取得し配列に追加、バイナリ形式でファイルに書き込む
    f.write(img2pdf.convert([Image.open(png_Folder+j).filename for j in natsorted(os.listdir(png_Folder))if j.endswith(extension)]))

In [360]:
path='./intan'
for pref in only_dir(path):
    c1=os.path.join(path,pref)
    for year in only_dir(c1):
        c2=os.path.join(c1,year)
        for tfile in natsorted(specific_get_ext(only_file(c2),'jpg')):
            pdf_FileName = f"./intan/{pref}/{year}/{pref}_{year}.pdf" # 出力するPDFの名前
            png_Folder = f"{c2}/" # 画像フォルダ
            extension  = ".jpg" # 拡張子がPNGのものを対象
            if len(natsorted(os.listdir(png_Folder)))>=2:
                with open(pdf_FileName,"wb") as f:
                    # 画像フォルダの中にあるPNGファイルを取得し配列に追加、バイナリ形式でファイルに書き込む
                    f.write(img2pdf.convert([Image.open(png_Folder+j).filename for j in natsorted(os.listdir(png_Folder))if j.endswith(extension) if j.startswith('t')]))

In [351]:
for pref in only_dir(path):
    c1=os.path.join(path,pref)
    for year in only_dir(c1):
        c2=os.path.join(c1,year)
        if len([i for i in natsorted(specific_get_ext(only_file(c2),'jpg')) if not i.endswith('.pdf')])==0:
            print(pref,year)

nagasaki 2021


# txtへ

In [ ]:
tools = pyocr.get_available_tools()
if len(tools) == 0:
    print("No OCR tool found")
    sys.exit(1)

# The tools are returned in the recommended order of usage
tool = tools[0]
print("Will use tool '%s'" % (tool.get_name()))
# Ex: Will use tool 'libtesseract'

In [ ]:
images = pdf2image.convert_from_path(test,fmt='jpg')
for i,image in enumerate(images):
    txt = tool.image_to_string(
        image,
        lang=lang,
        builder=pyocr.builders.TextBuilder()
    )
    f = open(f'./intan/hokkaido/2021/test{i+1}.txt', 'w') 
    # 文字列を記載する
    f.write(txt) 
    # ファイルを閉じる
    f.close() 

In [18]:
path='./intan'
for pref in tqdm(only_dir(path)):
    c1=os.path.join(path,pref)
    for year in only_dir(c1):
        c2=os.path.join(c1,year)
        _pdf=[i for i in os.listdir(c2) if i.endswith('.pdf')][0]
        images = pdf2image.convert_from_path(os.path.join(c2,_pdf),fmt='jpg')
        for i,image in enumerate(images):
            txt = tool.image_to_string(
                image,
                lang=lang,
                builder=pyocr.builders.TextBuilder()
            )
            f = open(f'./intan/{pref}/{year}/test{i+1}.txt', 'w') 
            # 文字列を記載する
            f.write(txt) 
            # ファイルを閉じる
            f.close() 

In [24]:
path='./intan'
for pref in tqdm(only_dir(path)):
    c1=os.path.join(path,pref)
    for year in only_dir(c1):
        c2=os.path.join(c1,year)
        if not len([i for i in os.listdir(c2) if i.startswith('t')if i.endswith('.jpg')])==len([i for i in os.listdir(c2) if i.endswith('.txt')]):
            print(pref,year)

hokkaido 2021



In [356]:
k1=[os.path.join(c2,i) for i in natsorted(os.listdir(c2)) if i.startswith('t')if i.endswith('.jpg')]

k2=[os.path.join(c2,i) for i in natsorted(os.listdir(c2)) if i.endswith('.txt')]

path='./intan'
c={}
for pref in tqdm(only_dir(path)):
    c1=os.path.join(path,pref)
    for year in only_dir(c1):
        c2=os.path.join(c1,year)
        k2=[os.path.join(c2,i) for i in natsorted(os.listdir(c2)) if i.startswith('t')if i.endswith('.jpg')]
        k1=[os.path.join(c2,i) for i in natsorted(os.listdir(c2)) if i.endswith('.txt')]
        for i,k in zip(k1,k2):
            c[i]=k

In [358]:
k2

['./intan/tochigi/2016/testtochigi_2016_01.jpg',
 './intan/tochigi/2016/testtochigi_2016_02.jpg',
 './intan/tochigi/2016/testtochigi_2016_03.jpg',
 './intan/tochigi/2016/testtochigi_2016_04.jpg',
 './intan/tochigi/2016/testtochigi_2016_05.jpg',
 './intan/tochigi/2016/testtochigi_2016_06.jpg',
 './intan/tochigi/2016/testtochigi_2016_07.jpg']

# 正規表現で抜き出す

In [48]:
import re

In [311]:
_str=k1[1]
_str

'./intan/tochigi/2016/test2.txt'

In [310]:
p = r'intan/(.*)/(\d)'

m = re.search(p, _str)
print(m.group(1))

tochigi


In [91]:
p=r'(\d{4})'

t = re.search(p, _str)
print(t.group(1))

In [93]:
def re_search(p,source):
    m = re.search(p, source)
    return m.group(1)

In [94]:
re_search(p,_str)

'2016'

In [325]:
p=r'/test(.*).txt'

d = re.search(p, _str)
print(d.group(1))

2


In [335]:
for pref in only_dir(path):
    c1=os.path.join(path,pref)
    for year in only_dir(c1):
        c2=os.path.join(c1,year)
        if len([tfile for tfile in natsorted(os.listdir(c2))if tfile.startswith('z')])==2:
            for tfile in natsorted(specific_get_ext(only_file(c2),'txt')):
                p2=r'/test(.*).txt'
                d = re_search(p2,os.path.join(c2,tfile))
                if int(d)<=0.5*len(natsorted(specific_get_ext(only_file(c2),'txt'))):
                    print(pref,d)

saitama 1
saitama 2
saitama 3
saitama 4
saitama 5
saitama 1
saitama 2
saitama 3
saitama 4
saitama 5
saitama 1
saitama 2
saitama 3
saitama 4
saitama 5
saitama 1
saitama 2
saitama 3
saitama 4
saitama 5
saitama 6
saitama 1
saitama 2
saitama 3
saitama 4
saitama 5
saitama 6
saitama 7
saitama 1
saitama 2
saitama 3
saitama 4
saitama 5
saitama 6
saitama 7
saitama 1
saitama 2
saitama 3
saitama 4
saitama 5
saitama 1
saitama 2
saitama 3
saitama 4
saitama 5
kochi 1
kochi 2
kochi 3
kochi 4
kochi 5
kochi 1
kochi 2
kochi 3
kochi 4
kochi 5
mie 1
mie 2
mie 3
mie 4
mie 5
mie 1
mie 2
mie 3
mie 4
mie 5
mie 1
mie 2
mie 3
mie 4
mie 5
mie 1
mie 2
mie 3
mie 4
mie 5
mie 1
mie 2
mie 3
mie 4
mie 5
mie 1
mie 2
mie 3
mie 4
mie 5
mie 1
mie 2
mie 3
mie 4
mie 5
saga 1
saga 2
saga 3
saga 4
saga 5
saga 6
saga 1
saga 2
saga 3
saga 4
saga 5
saga 6
saga 1
saga 2
saga 3
saga 4
saga 5
saga 6
saga 1
saga 2
saga 3
saga 4
saga 5
saga 6
saga 7
saga 1
saga 2
saga 3
saga 4
saga 5
saga 6
saga 7
saga 1
saga 2
saga 3
saga 4
saga 5
s

In [346]:
def get_key(my_dict,val):
    for key, value in my_dict.items():
         if val == value:
                return key
 
    return "There is no such Key"

In [347]:
get_key(c,'./intan/nagasaki/2014/testnagasaki_2014_01.jpg')

'./intan/nagasaki/2014/test1.txt'

In [ ]:
if any(x in txt for x in iden[bunya])

In [354]:
b=['試験管','ベru']
if any(x in txt for x in b):
    print('aiu')

aiu


In [350]:
txt

"Sさんは' ヒ トのだ液のはたらきにっいて調ベるため, 次の実験を行いま した。 さらに〟 消化に\nっいて調べました。 問ー~問4に答えなさい。 (20点)\n\n \n\n \n\n実験\n\n吹のー~6の順番で実験を行った。 図ー はそのよぅすを示したものでぁる。\n\nー デンプン ー gを水ー00cm‥二入れ` 加熱して藩かし, デンプン溶液をつくった。\n\n2 試験管A` 試験管Bを用意し ーでっ< ったデンプン溶液をそれぞれの試験管に5cm3\nずつ入れた。\n\n3 試験管Aにはうすめた自分のだ液ーcm3を, 試, 管Bには水ーcm3を加えてよく混ぜ合わせて\nから) それぞれの試験管を約4〇て に保った湯の中に ー。 分問入れた。\n\n4 ー。 分後, 試験管Aの液を試験管Cと試験管Dに半分ずつ入れ, 試験管Bの液を試験管E\nと試験管Fに半分ずつ入れた。\n\n5 試験管Cと試験管Eにヨゥ素液を数滴ずつ加え, それぞれの試験管中の液体の色の変化を\n観察した。 表ー は, その結果をまとめたものである。\n\n6 試験管Dと試験管Fにベネジク ト液ーcm~を加えてからぁる操作を行ぃ, それぞれの\n試験管中の液体の色の変化を観察した。 表2は, その結果をまとめたものである。\n\nデンプン溶液5cm〝ずつ\n\nうすめた3 ~\nだ液ーcm 水ーcm`\n\\ /\\ /\n\n冒\n\n  \n\n \n\n冨\n\nヨウ素液を ベネジク ト液を ヨウ素液を べネジク ト液を\n\n \n\n \n\n加える 加ぇる 加える 加える\n図ー\n表ー ョウ素液を加えた試験管 試験管C 試験管E\n試〟 管中の液体の色の変化 変化しなかった 青紫色に変化した\n\n \n\n \n\n表2 べネジク ト液を加えてから,\nぁる操作をした試験管\n\n試験管中の液体の色の変化 赤かっ色の誰譲ができた 変化しなかった\n\n試験管D 試験管F"

In [323]:
for pref in only_dir(path):
    c1=os.path.join(path,pref)
    for year in only_dir(c1):
        c2=os.path.join(c1,year)
        if len([tfile for tfile in natsorted(os.listdir(c2))if tfile.startswith('z')])==2:
            print(pref,year)
        print()
        print(len([tfile for tfile in natsorted(specific_get_ext(only_file(c2),'txt'))]))
            

saitama 2014
saitama 2015
saitama 2017
saitama 2019
saitama 2021
saitama 2020
saitama 2018
saitama 2016
kochi 2021
kochi 2018
mie 2014
mie 2017
mie 2019
mie 2021
mie 2020
mie 2018
mie 2016
saga 2014
saga 2015
saga 2017
saga 2019
saga 2021
saga 2020
saga 2018
saga 2016
osaka 2014
osaka 2015
osaka 2016
hiroshima 2015
hiroshima 2017
hiroshima 2019
hiroshima 2021
hiroshima 2020
hiroshima 2018
hiroshima 2016


# 県名

In [118]:
DATA='''北海道
青森県
岩手県
宮城県
秋田県
山形県
福島県
茨城県
栃木県
群馬県
埼玉県
千葉県
東京都
神奈川県
新潟県
富山県
石川県
福井県
山梨県
長野県
岐阜県
静岡県
愛知県
三重県
滋賀県
京都府
大阪府
兵庫県
奈良県
和歌山県
鳥取県
島根県
岡山県
広島県
山口県
徳島県
香川県
愛媛県
高知県
福岡県
佐賀県
長崎県
熊本県
大分県
宮崎県
鹿児島県
沖縄県
'''
data='''hokkaido
aomori
iwate
miyagi
akita
yamagata
fukushima
ibaraki
tochigi
gunma
saitama
chiba
tokyo
kanagawa
niigata
toyama
ishikawa
fukui
yamanashi
nagano
gifu
shizuoka
aichi
mie
shiga
kyoto
osaka
hyogo
nara
wakayama
tottori
shimane
okayama
hiroshima
yamaguchi
tokushima
kagawa
ehime
kochi
fukuoka
saga
nagasaki
kumamoto
oita
miyazaki
kagoshima
okinawa
'''

In [119]:
prefecture = re.findall(r'(\w+)\n',DATA)
_prefecture=re.findall(r'(\w+)\n',data)

pre_dict={}
for i,k in zip(prefecture,_prefecture):
    pre_dict[k]=i

In [254]:
def in_search(txtfile,bunya):
    f = open(txtfile, 'r')
    txt=f.read()
    if any(x in txt for x in iden[bunya]):
        return txtfile
#         A.append(txtfile)

In [259]:
def from_search_tolist(path,bunya):
    A=[]
    for pref in only_dir(path):
        c1=os.path.join(path,pref)
        for year in only_dir(c1):
            c2=os.path.join(c1,year)
            if sum(os.path.isfile(os.path.join(c2,name))for name in os.listdir(c2) if not name.startswith('.')if name.endswith('.txt'))>=1:
                for tfile in natsorted(specific_get_ext(only_file(c2),'txt')):
                    if not in_search(os.path.join(c2,tfile),bunya)==None:
                        A.append(in_search(os.path.join(c2,tfile),bunya))
    return A

In [261]:
len(from_search_tolist(path,'消化'))

77

In [249]:
def search_(txtfile,bunya):
    f = open(txtfile, 'r')
    txt=f.read()
    if any(x in txt for x in iden[bunya]):
        A.append(txtfile)

In [250]:
path='./intan'

A=[]
for pref in only_dir(path):
    c1=os.path.join(path,pref)
    for year in only_dir(c1):
        c2=os.path.join(c1,year)
        if sum(os.path.isfile(os.path.join(c2,name))for name in os.listdir(c2) if not name.startswith('.')if name.endswith('.txt'))>=1:
            for tfile in natsorted(specific_get_ext(only_file(c2),'txt')):
                search_(os.path.join(c2,tfile),'消化')

In [252]:
len(A)

77

In [262]:
bi='消化'

In [283]:
def in_search(txtfile,bunya):
    f = open(txtfile, 'r')
    txt=f.read()
    if any(x in txt for x in iden[bunya]):
        return txtfile
def key_from_search_tolist(path,keywoard):
    A=[]
    for pref in only_dir(path):
        c1=os.path.join(path,pref)
        for year in only_dir(c1):
            c2=os.path.join(c1,year)
            if sum(os.path.isfile(os.path.join(c2,name))for name in os.listdir(c2) if not name.startswith('.')if name.endswith('.txt'))>=1:
                for tfile in natsorted(specific_get_ext(only_file(c2),'txt')):
                    f = open(os.path.join(c2,tfile), 'r')
                    txt=f.read()
                    if keywoard in txt:
                        A.append(os.path.join(c2,tfile))
    return A

In [268]:
path='./intan'
A=from_search_tolist(path,bi)

In [284]:
keywoard='消化'

In [286]:
len(key_from_search_tolist(path,keywoard))

68

# 答えの出力

In [291]:
A[0]

'./intan/nagasaki/2014/test1.txt'

In [292]:
pref

'tochigi'

In [294]:
year

'2016'

In [300]:
os.path.join(os.path.join(path,pref,year),[i for i in os.listdir(os.path.join(path,pref,year))if i.startswith('z')][0])

'./intan/tochigi/2016/ztesttochigi_2016_01.jpg'

In [301]:
os.path.join(path,pref,year)

'./intan/tochigi/2016'

In [304]:
if len([i for i in os.listdir(os.path.join(path,pref,year))if i.startswith('z')])==2:
    print('aiu')

A[i]

# img size

In [408]:
import cv2

im = cv2.imread(testi)

In [411]:
for pref in only_dir(path):
        c1=os.path.join(path,pref)
        for year in only_dir(c1):
            c2=os.path.join(c1,year)
            k=[i for i in os.listdir(c2) if i.startswith('z')]
            im=cv2.imread(os.path.join(c2,k[0]))
            if not  im.shape[0]==2340:
                img=Image.open(os.path.join(c2,k[0]))
                img_resize=img.resize((1654,2340))
                img_resize.save('ztest{pref}_{year}_01.jpg')

In [421]:
for pref in only_dir(path):
        c1=os.path.join(path,pref)
        for year in only_dir(c1):
            c2=os.path.join(c1,year)
            k=[i for i in os.listdir(c2) if i.startswith('z')]
            im=cv2.imread(os.path.join(c2,k[0]))
            if not  im.shape[0]==2340:
                z=os.path.join(c2,k[0])
                Image.open(z).resize((1654,2340)).save(z)

In [415]:
Image.open(os.path.join(c2,k[0])).resize((1654,2340)).save('ztet{pref}_{year}_01.jpg')

In [416]:
Image.open(os.path.join(c2,k[0])).save('zte{pref}_{year}_01.jpg')

In [420]:
Image.open(os.path.join('./intan/tochigi/2014/ztesttochigi_2014_01.jpg')).resize((1654,2340)).save('f_01.jpg')

In [424]:
for pref in only_dir(path):
        c1=os.path.join(path,pref)
        for year in only_dir(c1):
            c2=os.path.join(c1,year)
            k=[i for i in os.listdir(c2) if i.endswith('.jpg')]
            im=cv2.imread(os.path.join(c2,k[0]))
            if not  im.shape[0]==2340:
                z=os.path.join(c2,k[0])
                Image.open(z).resize((1654,2340)).save(z)

In [425]:
for pref in only_dir(path):
        c1=os.path.join(path,pref)
        for year in only_dir(c1):
            c2=os.path.join(c1,year)
            k=[i for i in os.listdir(c2) if i.startswith('z')]
            im=cv2.imread(os.path.join(c2,k[0]))
            if not  im.shape[1]==1654:
                z=os.path.join(c2,k[0])
                Image.open(z).resize((1654,2340)).save(z)

In [437]:
for pref in only_dir(path):
    c1=os.path.join(path,pref)
    for year in only_dir(c1):
        c2=os.path.join(c1,year)
        for i in os.listdir(c2):
            if i.startswith('z'):
                if not cv2.imread(os.path.join(c2,i)).shape[0]==2340:
                    z=os.path.join(c2,i)
                    Image.open(z).resize((1654,2340)).save(z)

In [532]:
img=Image.open('test.jpg')
img_resize=img.resize((1654,2340))
img_resize.save('test.jpg')

In [433]:
for pref in only_dir(path):
    c1=os.path.join(path,pref)
    for year in only_dir(c1):
        c2=os.path.join(c1,year)
        print(c2)

./intan/nagasaki/2014
./intan/nagasaki/2015
./intan/nagasaki/2017
./intan/nagasaki/2019
./intan/nagasaki/2021
./intan/nagasaki/2020
./intan/nagasaki/2018
./intan/nagasaki/2016
./intan/nara/2015
./intan/nara/2017
./intan/nara/2019
./intan/nara/2021
./intan/nara/2020
./intan/nara/2018
./intan/nara/2016
./intan/saitama/2014
./intan/saitama/2015
./intan/saitama/2017
./intan/saitama/2019
./intan/saitama/2021
./intan/saitama/2020
./intan/saitama/2018
./intan/saitama/2016
./intan/oita/2014
./intan/oita/2015
./intan/oita/2017
./intan/oita/2019
./intan/oita/2021
./intan/oita/2020
./intan/oita/2018
./intan/oita/2016
./intan/ehime/2014
./intan/ehime/2017
./intan/ehime/2019
./intan/ehime/2021
./intan/ehime/2020
./intan/ehime/2018
./intan/ehime/2016
./intan/hokkaido/2014
./intan/hokkaido/2015
./intan/hokkaido/2017
./intan/hokkaido/2019
./intan/hokkaido/2021
./intan/hokkaido/2020
./intan/hokkaido/2018
./intan/hokkaido/2016
./intan/okinawa/2014
./intan/okinawa/2017
./intan/okinawa/2019
./intan/okinaw

# 白紙を入れる

In [499]:
def any_from_search_tolist(path,_list):
    A=[]
    for pref in only_dir(path):
        c1=os.path.join(path,pref)
        for year in only_dir(c1):
            c2=os.path.join(c1,year)
            if sum(os.path.isfile(os.path.join(c2,name))for name in os.listdir(c2) if not name.startswith('.')if name.endswith('.txt'))>=1:
                for tfile in natsorted(specific_get_ext(only_file(c2),'txt')):
                    f = open(os.path.join(c2,tfile), 'r')
                    txt=f.read()
                    if any(x in txt for x in b):
                        A.append(os.path.join(c2,tfile))
    return A

In [500]:
A=any_from_search_tolist(path,['消化','ベネジクト'])

In [501]:
A=[c[a] for a in A]

In [529]:
Af=[]
for i in range(len(A)-1):
    k=A[i]
    p = r'intan/(.*)/(\d)'
    pref_name=re_search(p,k)
    p1=r'(\d{4})'
    year_name=re_search(p1,k)
    
    k1=A[i+1]
    pref_1=re_search(p,k1)
    year_1=re_search(p1,k1)
    
    _path=os.path.join(path,pref_name,year_name)
    _path1=os.path.join(path,pref_1,year_1)
    Af.append(k)
    if not _path==_path1:
        if len([i for i in os.listdir(_path)if i.startswith('z')])==2:
            Af.append(os.path.join(_path,[i for i in os.listdir(_path)if i.startswith('z')][0]))
            Af.append(os.path.join(_path,[i for i in os.listdir(_path)if i.startswith('z')][1]))## 2個あったら2個入れる
        else:
            Af.append(os.path.join(_path,[i for i in natsorted(os.listdir(_path))if i.startswith('z')][0]))

In [530]:
def all_path():  
    c2_list=[]
    for pref in only_dir(path):
        c1=os.path.join(path,pref)
        for year in only_dir(c1):
            c2=os.path.join(c1,year)
            c2_list.append(c2)
    return c2_list

def path_par_num(c2_list):
    prefnum_zisyo={}
    for i in c2_list:
        if not len([t for t in A if i in t])==0:
            prefnum_zisyo[i]=len([t for t in A if i in t])+len([s for s in  os.listdir(i) if s.startswith('z')])
    return prefnum_zisyo

pdf='test.pdf'

##奇数の値を取り出す→リストに格納
def insert_kisuu(prefnum_zisyo,all_list):
    for k,v in prefnum_zisyo.items():
        if v%2==1:
            zjpeg_=[i for i in os.listdir(k)if i.startswith('z')][-1]
            all_list.insert(all_list.index(os.path.join(k,zjpeg_))+1,pdf)
    return all_list

Af=insert_kisuu(path_par_num(all_path()),Af)

In [531]:
Af

['./intan/nagasaki/2014/testnagasaki_2014_01.jpg',
 './intan/nagasaki/2014/testnagasaki_2014_08.jpg',
 './intan/nagasaki/2014/ztestnagasaki_2014_01.jpg',
 'test.pdf',
 './intan/nagasaki/2015/testnagasaki_2015_01.jpg',
 './intan/nagasaki/2015/testnagasaki_2015_03.jpg',
 './intan/nagasaki/2015/ztestnagasaki_2015_01.jpg',
 'test.pdf',
 './intan/nagasaki/2017/testnagasaki_2017_07.jpg',
 './intan/nagasaki/2017/ztestnagasaki_2017_01.jpg',
 './intan/nagasaki/2021/testsaitama_2021_05.jpg',
 './intan/nagasaki/2021/testsaitama_2021_07.jpg',
 './intan/nagasaki/2021/ztestnagasaki_2021_01.jpg',
 'test.pdf',
 './intan/nagasaki/2020/testnagasaki_2020_05.jpg',
 './intan/nagasaki/2020/ztestnagasaki_2020_01.jpg',
 './intan/nagasaki/2018/testnagasaki_2018_03.jpg',
 './intan/nagasaki/2018/testnagasaki_2018_07.jpg',
 './intan/nagasaki/2018/ztestnagasaki_2018_01.jpg',
 'test.pdf',
 './intan/nagasaki/2016/testnagasaki_2016_03.jpg',
 './intan/nagasaki/2016/ztestnagasaki_2016_01.jpg',
 './intan/nara/2015/testn

In [525]:
Af

['./intan/nagasaki/2014/testnagasaki_2014_01.jpg',
 './intan/nagasaki/2014/testnagasaki_2014_08.jpg',
 './intan/nagasaki/2014/ztestnagasaki_2014_01.jpg',
 './intan/nagasaki/2015/testnagasaki_2015_01.jpg',
 './intan/nagasaki/2015/testnagasaki_2015_03.jpg',
 './intan/nagasaki/2015/ztestnagasaki_2015_01.jpg',
 './intan/nagasaki/2017/testnagasaki_2017_07.jpg',
 './intan/nagasaki/2017/ztestnagasaki_2017_01.jpg',
 './intan/nagasaki/2021/testsaitama_2021_05.jpg',
 './intan/nagasaki/2021/testsaitama_2021_07.jpg',
 './intan/nagasaki/2021/ztestnagasaki_2021_01.jpg',
 './intan/nagasaki/2020/testnagasaki_2020_05.jpg',
 './intan/nagasaki/2020/ztestnagasaki_2020_01.jpg',
 './intan/nagasaki/2018/testnagasaki_2018_03.jpg',
 './intan/nagasaki/2018/testnagasaki_2018_07.jpg',
 './intan/nagasaki/2018/ztestnagasaki_2018_01.jpg',
 './intan/nagasaki/2016/testnagasaki_2016_03.jpg',
 './intan/nagasaki/2016/ztestnagasaki_2016_01.jpg',
 './intan/nara/2015/testnara_2015_02.jpg',
 './intan/nara/2015/ztestnara_2015

# 機械学習を用いた文書重要語カウント

In [537]:
from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.tokenfilter import POSStopFilter

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
import random

In [544]:
tokenizer = Tokenizer()
token_filters = [POSStopFilter(['記号','助詞','助動詞','動詞'])]
a = Analyzer(tokenizer=tokenizer, token_filters=token_filters)

In [547]:
test_tokens = a.analyze(txt)
for t in test_tokens:
    print(t)

(	名詞,サ変接続,*,*,*,*,(,*,*
一	名詞,数,*,*,*,*,一,イチ,イチ
)	名詞,サ変接続,*,*,*,*,),*,*
音	名詞,一般,*,*,*,*,音,オト,オト
光	名詞,一般,*,*,*,*,光,ヒカリ,ヒカリ
次	名詞,一般,*,*,*,*,次,ツギ,ツギ
ー	名詞,一般,*,*,*,*,ー,*,*
~	名詞,サ変接続,*,*,*,*,~,*,*
3	名詞,数,*,*,*,*,3,*,*
問い	名詞,一般,*,*,*,*,問い,トイ,トイ
な	形容詞,自立,*,*,形容詞・アウオ段,ガル接続,ない,ナ,ナ
さ	名詞,接尾,特殊,*,*,*,さ,サ,サ
ぃ	名詞,一般,*,*,*,*,ぃ,*,*
ー	名詞,一般,*,*,*,*,ー,*,*
花火	名詞,一般,*,*,*,*,花火,ハナビ,ハナビ
開	名詞,固有名詞,一般,*,*,*,開,ヒラキ,ヒラキ
ぃた	名詞,一般,*,*,*,*,ぃた,*,*
瞬	名詞,固有名詞,人名,名,*,*,瞬,マドカ,マドカ
聞	名詞,一般,*,*,*,*,聞,*,*
,	名詞,サ変接続,*,*,*,*,,,*,*
その	連体詞,*,*,*,*,*,その,ソノ,ソノ
花火	名詞,一般,*,*,*,*,花火,ハナビ,ハナビ
光	名詞,一般,*,*,*,*,光,ヒカリ,ヒカリ
音	名詞,一般,*,*,*,*,音,オト,オト
同時に	副詞,一般,*,*,*,*,同時に,ドウジニ,ドージニ
発生	名詞,サ変接続,*,*,*,*,発生,ハッセイ,ハッセイ
太郎	名詞,固有名詞,人名,名,*,*,太郎,タロウ,タロー
さん	名詞,接尾,人名,*,*,*,さん,サン,サン
地	名詞,一般,*,*,*,*,地,チ,チ
点	名詞,一般,*,*,*,*,点,テン,テン
,	名詞,サ変接続,*,*,*,*,,,*,*
その	連体詞,*,*,*,*,*,その,ソノ,ソノ
花火	名詞,一般,*,*,*,*,花火,ハナビ,ハナビ
光	名詞,一般,*,*,*,*,光,ヒカリ,ヒカリ
ぇてしぱらく	名詞,一般,*,*,*,*,ぇてしぱらく,*,*
,	名詞,サ変接続,*,*,*,*,,,*,*
その	連体詞,*,*,*,*,*,その,ソノ,ソノ
音	名詞,一般,*,*,*,*,音,オト,オト
(	名詞,サ変接続,*,*,*,*,(,*,*

In [549]:
texts_flat= "。".join(txt)
tokens = a.analyze(texts_flat)
_txt= ' '.join([t.surface for t in tokens])

In [105]:
import MeCab
import re
import pandas as pd
from collections import Counter

In [90]:
txt_list=[os.path.join('./intan/nagasaki/2015',i) for i in natsorted(os.listdir('./intan/nagasaki/2015')) if i.endswith('.txt')]

In [235]:
def to_mecab(txt):##名詞を引き抜いて単語ごとに空白あるような文字列を返す  ##数字落としたい
    m = MeCab.Tagger()
    node = m.parseToNode(txt)
    words=[]
    while node:
        f=node.feature
        p=f.split(',')[0]
        if p=='名詞':
            words.append(node.surface)
        node=node.next
    return ' '.join(words)

def open_txt(path):#txtfileを引き抜く
    with open(path) as f:
        return f.read()

def each_txt_to_mecab(txt_path_list):##それぞれのテキストをlistへ
    docs=[]
    for i in txt_path_list:
        text=open_txt(i)
        text_meisi=to_mecab(text)
        docs.append(text_meisi)
    return docs

def mecab_to_array(txt_path_list):## listを分析してarrayで返す
    docs=each_txt_to_mecab(txt_path_list)
    vectorizer = TfidfVectorizer(smooth_idf=False)
    X = vectorizer.fit_transform(docs)
    values = X.toarray()
    return values

def visualize_df(txt_path_list):
    docs=each_txt_to_mecab(txt_path_list)
    vectorizer = TfidfVectorizer(smooth_idf=False)
    X = vectorizer.fit_transform(docs)
    values=mecab_to_array(txt_path_list)
    feature_names = vectorizer.get_feature_names()
    df = pd.DataFrame(values, columns = feature_names)
    return df

def cossim(v1,v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

類似度テスト

In [121]:
a=["./intan/nagasaki/2014/test1.txt"
,"./intan/nagasaki/2020/test5.txt"
,"./intan/nara/2016/test3.txt"]


In [212]:
visualize_df(txt_list)[5:6].T.sort_values(by=5,ascending=False).head(10)

,5
物体,0.866095
水平,0.301250
通過,0.150625
面上,0.150625
運動,0.145901
平行,0.112969
静止,0.112969
摩擦,0.112969
とき,0.107854
ばねばかり,0.075313


上位の10個が特定の

In [214]:
visualize_df(txt_list)[5:6].T.sort_values(by=5,ascending=False).head(10).index.to_list()

['物体', '水平', '通過', '面上', '運動', '平行', '静止', '摩擦', 'とき', 'ばねばかり']

In [300]:
path='./intan/nagasaki'

In [365]:
_path='./intan'

In [366]:
for pref in natsorted([i for i in os.listdir(_path)if not i.startswith('.')]):
    path_pref=os.path.join(_path,pref)
    for i in natsorted([i for i in os.listdir(path_pref)if not i.startswith('.')]):
        path_year=os.path.join(path_pref,i)
        txt_list=[os.path.join(path_year,i) for i in natsorted(os.listdir(path_year))if i.endswith('.txt')]
        for txt in range(len(txt_list)):
            _list=visualize_df(txt_list)[txt:txt+1].T.sort_values(by=txt,ascending=False).head(50).index.to_list()
            if any(t in _list for t in hikari):
                print(pref,i,txt+1)

ehime 2014 1
ehime 2017 1
ehime 2020 1
fukui 2017 4
fukui 2020 5
gifu 2014 6
gunma 2015 1
gunma 2016 7
gunma 2019 3
hiroshima 2017 1
hiroshima 2017 6
hiroshima 2017 8
hiroshima 2017 9
hiroshima 2017 12
hokkaido 2014 10
hokkaido 2020 1
hyogo 2015 7
kagoshima 2014 3
kagoshima 2016 5
kagoshima 2018 7
kagoshima 2020 6
kagoshima 2021 1
kanagawa 2015 1
kanagawa 2015 2
kanagawa 2015 5
kanagawa 2016 1
kanagawa 2021 2
kochi 2017 3
kochi 2017 4
kochi 2017 7
kochi 2021 4
kyoto 2016 4
mie 2019 9
mie 2019 10
miyazaki 2017 3
miyazaki 2017 4
miyazaki 2021 8
nagano 2019 8
nagasaki 2017 2
nagasaki 2019 2
nagasaki 2021 6
nara 2021 5
nara 2021 6
oita 2015 7
okinawa 2019 5
okinawa 2019 6
okinawa 2019 14
okinawa 2021 1
okinawa 2021 7
osaka 2021 6
saga 2018 1
saga 2018 9
saga 2018 12
saga 2021 2
saga 2021 9
saga 2021 10
saga 2021 11
saga 2021 14
saitama 2016 1
saitama 2016 7
saitama 2016 9
saitama 2016 10
shiga 2014 2
shizuoka 2021 4
shizuoka 2021 7
tochigi 2014 3
tochigi 2018 6
tochigi 2021 7
tokyo 2016 2


In [364]:
for i in natsorted([i for i in os.listdir(path)if not i.startswith('.')]):
    path_year=os.path.join(path,i)
    txt_list=[os.path.join(path_year,i) for i in natsorted(os.listdir(path_year))if i.endswith('.txt')]
    for txt in range(len(txt_list)):
        _list=visualize_df(txt_list)[txt:txt+1].T.sort_values(by=txt,ascending=False).head(50).index.to_list()
        if any(t in _list for t in hikari):
            print(i,txt+1)

2017 2
2019 2
2021 6


In [237]:
visualize_df(txt_list)[5:6].T.sort_values(by=5,ascending=False).head(10).index.to_list()

['スクリーン', '入射', '容器', '物体', '硬貨', '境界', '矢印', '光源', 'レンズ', 'ところ']

In [363]:
hikari=['スクリーン','光','焦点','入射']
tisou=['はんれい岩','地層']
ボーリング

In [242]:
if any(t in _list for t in x):
    print('aiu')

aiu


In [367]:
tt='./intan/saitama/2021'

In [368]:
path_year=tt

In [369]:
txt_list=[os.path.join(path_year,i) for i in natsorted(os.listdir(path_year))if i.endswith('.txt')]

In [370]:
txt_list

['./intan/saitama/2021/test1.txt',
 './intan/saitama/2021/test2.txt',
 './intan/saitama/2021/test3.txt',
 './intan/saitama/2021/test4.txt',
 './intan/saitama/2021/test5.txt',
 './intan/saitama/2021/test6.txt',
 './intan/saitama/2021/test7.txt',
 './intan/saitama/2021/test8.txt',
 './intan/saitama/2021/test9.txt',
 './intan/saitama/2021/test10.txt',
 './intan/saitama/2021/test11.txt',
 './intan/saitama/2021/test12.txt',
 './intan/saitama/2021/test13.txt',
 './intan/saitama/2021/test14.txt']

In [384]:
mecab_to_array(txt_list)

cossim(mecab_to_array(txt_list)[3],mecab_to_array(txt_list)[4])

In [390]:
for i in range(len(txt_list)-1):
    if cossim(mecab_to_array(txt_list)[i],mecab_to_array(txt_list)[i+1])>=0.05:
        print(i+1,i+2,cossim(mecab_to_array(txt_list)[i],mecab_to_array(txt_list)[i+1]))

1 2 0.1401951593644004
3 4 0.05316894836761645
4 5 0.17432527437230078
6 7 0.20104179468739278
7 8 0.0750318605867533
9 10 0.18922759465193295
10 11 0.31680234000085
11 12 0.06152892798698996
12 13 0.5410038903092224
13 14 0.15030219784555082


In [397]:
path='./intan'

In [411]:
def only_dir(path):
    return [i for i in os.listdir(path)if os.path.isdir(os.path.join(path,i))if not i.startswith('.')]
def only_file(path):
    return [i for i in os.listdir(path)if os.path.isfile(os.path.join(path,i))if not i.startswith('.')]
def specific_get_ext(_list,extension):
        return [i for i in _list if i.endswith(f'.{extension}') ]
def get_key(my_dict,val):
    for key, value in my_dict.items():
         if val == value:
                return key
def any_from_search_tolist(path,_list):
    A=[]
    for pref in only_dir(path):
        c1=os.path.join(path,pref)
        for year in only_dir(c1):
            c2=os.path.join(c1,year)
            if sum(os.path.isfile(os.path.join(c2,name))for name in os.listdir(c2) if not name.startswith('.')if name.endswith('.txt'))>=1:
                for tfile in natsorted(specific_get_ext(only_file(c2),'txt')):
                    f = open(os.path.join(c2,tfile), 'r')
                    txt=f.read()
                    if any(x in txt for x in _list):
                        A.append(os.path.join(c2,tfile))
    return A
def re_search(p,source):
    m = re.search(p, source)
    return m.group(1)

In [403]:
b=['消化']

In [407]:
syouka=any_from_search_tolist(path,b)

In [408]:
syouka

['./intan/nagasaki/2014/test1.txt',
 './intan/nagasaki/2020/test5.txt',
 './intan/nara/2016/test3.txt',
 './intan/saitama/2014/test5.txt',
 './intan/saitama/2014/test6.txt',
 './intan/oita/2020/test1.txt',
 './intan/oita/2020/test2.txt',
 './intan/ehime/2018/test3.txt',
 './intan/hokkaido/2015/test5.txt',
 './intan/hokkaido/2015/test6.txt',
 './intan/hokkaido/2021/test3.txt',
 './intan/hokkaido/2021/test4.txt',
 './intan/okinawa/2014/test5.txt',
 './intan/okinawa/2014/test6.txt',
 './intan/okinawa/2019/test8.txt',
 './intan/okinawa/2021/test10.txt',
 './intan/kochi/2017/test5.txt',
 './intan/kochi/2017/test6.txt',
 './intan/fukui/2014/test1.txt',
 './intan/fukui/2020/test1.txt',
 './intan/wakayama/2019/test2.txt',
 './intan/yamaguchi/2021/test10.txt',
 './intan/yamaguchi/2020/test9.txt',
 './intan/yamaguchi/2016/test8.txt',
 './intan/tokyo/2017/test8.txt',
 './intan/tokyo/2021/test1.txt',
 './intan/tokyo/2020/test7.txt',
 './intan/tokyo/2020/test8.txt',
 './intan/hyogo/2019/test2.txt',

In [425]:
syouka[15]

'./intan/okinawa/2021/test10.txt'

In [416]:
p = r'intan/(.*)/(\d)'
pref_name=re_search(p,syouka[0])

In [414]:
pref_name


'nagasaki'

In [494]:
p = r'/test(\d{1,2}).txt'

In [495]:
pref_name=re_search(p,syouka[15])

In [497]:
int(pref_name)

10

In [502]:
syouka[0]

'./intan/nagasaki/2014/test1.txt'

In [ ]:
pref_name=re_search(p1,i)

In [511]:
p1=r'intan/(.*)/(\d)'
p2=r'/(\d{4})'
p3= r'/test(\d{1,2}).txt'
_path='./intan'
for i in syouka:
    page=re_search(p3,i)
    pref_name=re_search(p1,i)
    year=re_search(p2,i)
    path=os.path.join(_path,pref_name,year)
    txt_list=natsorted([os.path.join(path,i) for i in os.listdir(path)if i.endswith('.txt')])
    try:
        if cossim(mecab_to_array(txt_list)[int(page)-1],mecab_to_array(txt_list)[int(page)])>=0.05:
            print(cossim(mecab_to_array(txt_list)[int(page)-1],mecab_to_array(txt_list)[int(page)]))
            print(i)
    except:
        print('miss:',i)

0.263531030070939
./intan/nara/2016/test3.txt
0.053955390646287356
./intan/saitama/2014/test5.txt
0.272550338200659
./intan/oita/2020/test1.txt
0.07139974476842909
./intan/ehime/2018/test3.txt
0.22189307909176792
./intan/hokkaido/2015/test5.txt
0.6415980073055314
./intan/hokkaido/2021/test3.txt
0.10778047184259466
./intan/hokkaido/2021/test4.txt
0.20876436101516196
./intan/okinawa/2014/test5.txt
0.11745309510970313
./intan/okinawa/2014/test6.txt
0.13737168460715043
./intan/kochi/2017/test5.txt
0.08003596125614576
./intan/fukui/2014/test1.txt
miss: ./intan/yamaguchi/2021/test10.txt
0.23718885683555788
./intan/yamaguchi/2020/test9.txt
0.12534088784020944
./intan/tokyo/2017/test8.txt
0.687312810385101
./intan/tokyo/2020/test7.txt
0.0999928458927022
./intan/tokyo/2020/test8.txt
0.11825323225350692
./intan/hyogo/2019/test2.txt
0.1743966922143843
./intan/hyogo/2016/test5.txt
0.34767560161187205
./intan/mie/2016/test3.txt
0.27002709692447424
./intan/nagano/2021/test1.txt
0.2652752725160292
./

In [513]:
for i in range(19-1):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


In [516]:
path='./intan'

In [517]:
def all_path():  
    c2_list=[]
    for pref in only_dir(path):
        c1=os.path.join(path,pref)
        for year in only_dir(c1):
            c2=os.path.join(c1,year)
            c2_list.append(c2)
    return c2_list

def path_par_num(c2_list):
    prefnum_zisyo={}
    for i in c2_list:
        if not len([t for t in A if i in t])==0:
            prefnum_zisyo[i]=len([t for t in A if i in t])+len([s for s in  os.listdir(i) if s.startswith('z')])
    return prefnum_zisyo

In [519]:
path_par_num(all_path())

{}

In [520]:
test='回 地震にっぃて、 あとの問いに答えなさい。地震は、 主にプレー トの動きにょって引き起こされると考えられている。 プレー トの動きにょって大地をっく ってぃる岩石に力が加わり、 その力にたえきれなく なった岩石が破壊され大地がずれゑと地震が発生する。問ー 日本列島付近にあるユーラシアプレー ト、 北アメ リカプレー ト、 太平洋プレー ト、 フィ リ ピン海プレートのうち、 海洋プレート 〈海のフ〝レート) である太平洋プレートとフィ リピン海プレートの動く向きとして最も適当なものは、 次のどれか。教肯委員会かちの提供によ り問題の訂正を赤囲み部分で行ぃま Lた。問2 下線部によって生じる 「大地のずれ」 を何というか。問3 震源の真上の地表の点を何というか。問4 図は、 ある場所で発生した地震にっいて、 震源からの距離と P波と S波がそれぞれ届く までの時聞の関係を表したものである。 ただし、 P波と S波はそれぞれ一定の速さで伝わるものと'

In [564]:
if any(x in txt for x in ['火成','流紋','安山','玄武','班晶','石基','マグマ','溶岩','火山ガス','桜島']):
    print('aiu')

In [559]:
pa='./intan/nagasaki/2016/test3.txt'

In [560]:
with open(pa) as f:
    txt=f.read()

In [561]:
txt

'回 次のー、 立の問いに答えなさい。\n\nー 白い粉末状の物質A~Cがあり、 それぞれは、 食塩、 砂糖、 重そう (炭酸水素ナ ト リ ウム) のい\nずれかである。 これらを区別するために実験ー を行った。\n\n【実験ー】\n\n手順ー 物質A~Cをそれぞれ別の燃焼皿にと り、 図 ー のようにヵ口熱して、\n物質の様子を観察した。 物質Aはこげて黒く なったが、 物質 B、 Cは\n加熱後も白色であった。\n\n手順2 新たに物質B、 Cをそれぞれ別の試験管に ー・0gずつと り、 室温で水5・0mL を加えてょ\nく振り、 試験管内の様子を観察しだ。 物質 Bはすぺて溶けたが、 物質Cは一部が溶け残った。\n\n手順3 手順2のそれぞれの試験管にフェノールフタ レイ ン溶液を数滴加え、 観察した。 物質 Bの\n水溶液は無色のままであったが、 物質Cの水溶液は薄い赤色になった。\n\n \n\n問ー 物質A~Cは何か、 それぞれの名称を答えよ。\n\n問2 物質Aにっいても手順2の操作を行う と、 すぺて溶けて透明な水溶液になった。 図2は溶け\nて透明になった直後の水溶液中の溶質の分布をあらわしたモデル図である。 この水溶液を、 数\n時聞放置した後の溶質の分布をあらわしたモデル図と して、 最も適当なものは、 次のどれか。\n図2 イ ウ\n\n   \n\n立 鉄粉と硫黄の粉末をょ く混ぜた混合物をっく り、 じゅうぷんに換気をしながら実験2を行った。\n\n【実験2】 混合物を試験管に入れ、 図3のように混合物の上部を加熱すると反応が始まった。 加熱を\nゃめても光と熱を発しながら反応は進み、 黒色の物質ができた。\n\n問3 実験2で、 鉄粉と硫黄の粉末から黒色の物質が生じる反応を、 化学\n反応式で書け。\n\n問4 鉄粉2.8 g と硫黄の粉末2.0 g を反応させ、 完全に反応が進んだと\nきに生成した黒色の物質の質量と して、 最も適当なものは、 次のどれ\nか。 ただ し、 鉄と硫黄は質量比フ ‥ 4で反応するものとする。\nア 4.4g イ 4.8g ゥ 5.5g エ ーーg\n\n \n\n問5 実験2の下線部のょうに熱を発する反応を発熱反応とぃう。 次のa~cの操作で起こる反応\nのうち、 発熱反応の組み合わせと して、 最も

In [566]:
path

'./intan'

In [577]:
A=any_from_search_tolist(path,'消化')

In [579]:
A

['./intan/nagasaki/2014/test1.txt',
 './intan/nagasaki/2014/test3.txt',
 './intan/nagasaki/2014/test4.txt',
 './intan/nagasaki/2014/test6.txt',
 './intan/nagasaki/2014/test7.txt',
 './intan/nagasaki/2014/test8.txt',
 './intan/nagasaki/2015/test2.txt',
 './intan/nagasaki/2015/test3.txt',
 './intan/nagasaki/2015/test5.txt',
 './intan/nagasaki/2015/test7.txt',
 './intan/nagasaki/2015/test8.txt',
 './intan/nagasaki/2017/test1.txt',
 './intan/nagasaki/2017/test3.txt',
 './intan/nagasaki/2017/test5.txt',
 './intan/nagasaki/2017/test6.txt',
 './intan/nagasaki/2017/test7.txt',
 './intan/nagasaki/2017/test8.txt',
 './intan/nagasaki/2019/test1.txt',
 './intan/nagasaki/2019/test3.txt',
 './intan/nagasaki/2019/test4.txt',
 './intan/nagasaki/2019/test6.txt',
 './intan/nagasaki/2019/test7.txt',
 './intan/nagasaki/2021/test1.txt',
 './intan/nagasaki/2021/test3.txt',
 './intan/nagasaki/2021/test7.txt',
 './intan/nagasaki/2021/test8.txt',
 './intan/nagasaki/2020/test3.txt',
 './intan/nagasaki/2020/test

In [569]:
c={}
for i in A:
    c[i]=[i]

In [578]:
random.sample(A,len(A))

['./intan/tokyo/2018/test3.txt',
 './intan/shizuoka/2016/test3.txt',
 './intan/okinawa/2017/test5.txt',
 './intan/kochi/2016/test7.txt',
 './intan/okinawa/2018/test5.txt',
 './intan/hokkaido/2014/test4.txt',
 './intan/hyogo/2020/test1.txt',
 './intan/nara/2016/test4.txt',
 './intan/hyogo/2020/test7.txt',
 './intan/saga/2020/test11.txt',
 './intan/gifu/2021/test5.txt',
 './intan/yamaguchi/2021/test2.txt',
 './intan/yamaguchi/2017/test4.txt',
 './intan/miyazaki/2017/test5.txt',
 './intan/hokkaido/2015/test4.txt',
 './intan/shiga/2020/test1.txt',
 './intan/gunma/2016/test7.txt',
 './intan/osaka/2018/test8.txt',
 './intan/mie/2017/test7.txt',
 './intan/toyama/2014/test6.txt',
 './intan/nagasaki/2021/test1.txt',
 './intan/gunma/2014/test6.txt',
 './intan/nagasaki/2014/test1.txt',
 './intan/kagoshima/2017/test5.txt',
 './intan/toyama/2015/test6.txt',
 './intan/miyazaki/2014/test2.txt',
 './intan/saitama/2015/test8.txt',
 './intan/miyazaki/2016/test4.txt',
 './intan/saitama/2019/test13.txt',


In [ ]:
if './intan'

In [580]:
for i in A:
    if './intan/nagasaki/2015' in i:
        print(i)

./intan/nagasaki/2015/test2.txt
./intan/nagasaki/2015/test3.txt
./intan/nagasaki/2015/test5.txt
./intan/nagasaki/2015/test7.txt
./intan/nagasaki/2015/test8.txt


In [589]:
A

['./intan/nagasaki/2014/test1.txt',
 './intan/nagasaki/2014/test3.txt',
 './intan/nagasaki/2014/test4.txt',
 './intan/nagasaki/2014/test6.txt',
 './intan/nagasaki/2014/test7.txt',
 './intan/nagasaki/2014/test8.txt',
 './intan/nagasaki/2015/test2.txt',
 './intan/nagasaki/2015/test3.txt',
 './intan/nagasaki/2015/test5.txt',
 './intan/nagasaki/2015/test7.txt',
 './intan/nagasaki/2015/test8.txt',
 './intan/nagasaki/2017/test1.txt',
 './intan/nagasaki/2017/test3.txt',
 './intan/nagasaki/2017/test5.txt',
 './intan/nagasaki/2017/test6.txt',
 './intan/nagasaki/2017/test7.txt',
 './intan/nagasaki/2017/test8.txt',
 './intan/nagasaki/2019/test1.txt',
 './intan/nagasaki/2019/test3.txt',
 './intan/nagasaki/2019/test4.txt',
 './intan/nagasaki/2019/test6.txt',
 './intan/nagasaki/2019/test7.txt',
 './intan/nagasaki/2021/test1.txt',
 './intan/nagasaki/2021/test3.txt',
 './intan/nagasaki/2021/test7.txt',
 './intan/nagasaki/2021/test8.txt',
 './intan/nagasaki/2020/test3.txt',
 './intan/nagasaki/2020/test

In [622]:
def dict_path_list(A):
##pathをuniqueに抜き出してくる
    p1=r'intan/(.*)/(\d)'
    p2=r'/(\d{4})'
    path_list=[]
    for i in A:
        pref_name=re_search(p1,i)
        year=re_search(p2,i)
        path_=os.path.join(path,pref_name,year)
        path_list.append(path_)
    return path_list

def zisyo_unique_num(path_list):
    zisyo={}
    for i in list(set(path_list)):
        k=[]
        for a in A:
            if i in a:
                k.append(a)
        zisyo[i]=natsorted(k)
    return zisyo

def new_random(num):
    A_b=[]
    p1=r'intan/(.*)/(\d)'
    p2=r'/(\d{4})'
    A_random=random.sample(A,num)
    for i in A_random:
        pref_name=re_search(p1,i)
        year=re_search(p2,i)
        path_=os.path.join(path,pref_name,year)
        if not i in A_b: 
            for z in zisyo_unique_num(dict_path_list(A))[path_]:
                A_b.append(z)
    return A_b

In [617]:
A_b

['./intan/hokkaido/2021/test1.txt',
 './intan/hokkaido/2021/test3.txt',
 './intan/hokkaido/2021/test4.txt',
 './intan/hokkaido/2021/test5.txt',
 './intan/hokkaido/2021/test6.txt',
 './intan/wakayama/2018/test1.txt',
 './intan/wakayama/2018/test2.txt',
 './intan/wakayama/2018/test3.txt',
 './intan/wakayama/2018/test5.txt',
 './intan/wakayama/2018/test6.txt',
 './intan/wakayama/2018/test7.txt',
 './intan/wakayama/2018/test8.txt',
 './intan/tokyo/2017/test1.txt',
 './intan/tokyo/2017/test4.txt',
 './intan/tokyo/2017/test5.txt',
 './intan/tokyo/2017/test6.txt',
 './intan/tokyo/2017/test7.txt',
 './intan/tokyo/2017/test8.txt',
 './intan/tokyo/2017/test9.txt',
 './intan/yamaguchi/2016/test2.txt',
 './intan/yamaguchi/2016/test4.txt',
 './intan/yamaguchi/2016/test8.txt',
 './intan/fukui/2019/test1.txt',
 './intan/fukui/2019/test2.txt',
 './intan/fukui/2019/test3.txt',
 './intan/fukui/2019/test4.txt',
 './intan/fukui/2019/test5.txt',
 './intan/nara/2016/test1.txt',
 './intan/nara/2016/test2.txt

In [621]:
zisyo_unique_num(dict_path_list(A))

{'./intan/shizuoka/2015': ['./intan/shizuoka/2015/test1.txt',
  './intan/shizuoka/2015/test2.txt',
  './intan/shizuoka/2015/test3.txt',
  './intan/shizuoka/2015/test4.txt',
  './intan/shizuoka/2015/test5.txt',
  './intan/shizuoka/2015/test6.txt'],
 './intan/gunma/2017': ['./intan/gunma/2017/test1.txt',
  './intan/gunma/2017/test2.txt',
  './intan/gunma/2017/test5.txt',
  './intan/gunma/2017/test6.txt'],
 './intan/tokyo/2018': ['./intan/tokyo/2018/test2.txt',
  './intan/tokyo/2018/test3.txt',
  './intan/tokyo/2018/test4.txt',
  './intan/tokyo/2018/test5.txt',
  './intan/tokyo/2018/test6.txt',
  './intan/tokyo/2018/test8.txt',
  './intan/tokyo/2018/test9.txt',
  './intan/tokyo/2018/test10.txt'],
 './intan/osaka/2019': ['./intan/osaka/2019/test1.txt',
  './intan/osaka/2019/test2.txt',
  './intan/osaka/2019/test3.txt',
  './intan/osaka/2019/test4.txt',
  './intan/osaka/2019/test5.txt',
  './intan/osaka/2019/test6.txt',
  './intan/osaka/2019/test8.txt'],
 './intan/kanagawa/2020': ['./intan/

In [614]:
zisyo

{'./intan/shizuoka/2015': ['./intan/shizuoka/2015/test1.txt',
  './intan/shizuoka/2015/test2.txt',
  './intan/shizuoka/2015/test3.txt',
  './intan/shizuoka/2015/test4.txt',
  './intan/shizuoka/2015/test5.txt',
  './intan/shizuoka/2015/test6.txt'],
 './intan/gunma/2017': ['./intan/gunma/2017/test1.txt',
  './intan/gunma/2017/test2.txt',
  './intan/gunma/2017/test5.txt',
  './intan/gunma/2017/test6.txt'],
 './intan/tokyo/2018': ['./intan/tokyo/2018/test2.txt',
  './intan/tokyo/2018/test3.txt',
  './intan/tokyo/2018/test4.txt',
  './intan/tokyo/2018/test5.txt',
  './intan/tokyo/2018/test6.txt',
  './intan/tokyo/2018/test8.txt',
  './intan/tokyo/2018/test9.txt',
  './intan/tokyo/2018/test10.txt'],
 './intan/osaka/2019': ['./intan/osaka/2019/test1.txt',
  './intan/osaka/2019/test2.txt',
  './intan/osaka/2019/test3.txt',
  './intan/osaka/2019/test4.txt',
  './intan/osaka/2019/test5.txt',
  './intan/osaka/2019/test6.txt',
  './intan/osaka/2019/test8.txt'],
 './intan/kanagawa/2020': ['./intan/

In [615]:
zisyo[path_]

['./intan/tochigi/2016/test1.txt',
 './intan/tochigi/2016/test4.txt',
 './intan/tochigi/2016/test6.txt',
 './intan/tochigi/2016/test7.txt']

In [607]:
c={}

In [608]:
c['zisyo']=[1,2]

In [606]:
c

{'zisyo': [1, 2]}

In [600]:
natsorted(k)

['./intan/toyama/2016/test2.txt',
 './intan/toyama/2016/test3.txt',
 './intan/toyama/2016/test4.txt',
 './intan/toyama/2016/test5.txt']

In [ ]:
for i in 

In [596]:
[1,3,4,5,(2,3)]

[1, 3, 4, 5, (2, 3)]

In [575]:
p1=r'intan/(.*)/(\d)'
p2=r'/(\d{4})'
for i in range(len(A)-1):
    k=A[i]
    k1=A[i+1]
    pref_name=re_search(p1,k)
    year=re_search(p2,k)
    if 
#     pref_name1=re_search(p1,k1)
#     year1=re_search(p2,k1)
#     _path=os.path.join(path,pref_name,year)
#     _path1=os.path.join(path,pref_name1,year1)
#     if _path==_path1:
#         print('similary')
#     else:
#         print('none')

similary
similary
similary
similary
similary
none
similary
similary
similary
similary
none
similary
similary
similary
similary
similary
none
similary
similary
similary
similary
none
similary
similary
similary
none
similary
similary
similary
similary
none
similary
similary
similary
none
similary
similary
similary
similary
none
similary
similary
similary
none
similary
similary
none
similary
similary
similary
none
similary
similary
similary
similary
similary
none
similary
similary
similary
similary
none
similary
similary
none
similary
similary
similary
none
similary
similary
similary
similary
similary
similary
similary
none
similary
similary
similary
similary
similary
none
similary
similary
similary
none
similary
similary
similary
similary
similary
similary
none
similary
similary
similary
similary
similary
similary
similary
none
similary
similary
similary
similary
similary
similary
similary
none
similary
similary
similary
similary
similary
similary
none
similary
similary
similary
similary